In [1]:
import sys
sys.path.append('..')
from run_fft import FFTProcessor
import numpy as np
import pandas as pd

In [2]:
# Enable rpy2
%load_ext rpy2.ipython

In [3]:
fft_processor = FFTProcessor(method='fft', 
                             preprocess='logzs', 
                             value='norm', 
                             require_sid=False)

Read NLL data, plot density; show how `log` and `z-score` affect the distribution.

In [11]:
nll_pubmed_orig = fft_processor._read_data(data_file='../data/gpt-4/pubmed_gpt-4.original.mistral.nll.txt')
nll_pubmed_samp = fft_processor._read_data(data_file='../data/gpt-4/pubmed_gpt-4.sampled.mistral.nll.txt')

print(len(nll_pubmed_orig), len(nll_pubmed_samp))
print(type(nll_pubmed_orig), type(nll_pubmed_samp))

# Raw NLL
df_nll_orig = fft_processor._create_input_df(nll_pubmed_orig)
df_nll_samp = fft_processor._create_input_df(nll_pubmed_samp)
df_nll_orig['Source'] = 'Human'
df_nll_samp['Source'] = 'GPT-4'
df_nll = pd.concat([df_nll_orig, df_nll_samp])

# Log NLL
fft_processor.preprocess = 'log'
data = fft_processor._preprocess(nll_pubmed_orig)
df_nlllog_orig = fft_processor._create_input_df(data)
data = fft_processor._preprocess(nll_pubmed_samp)
df_nlllog_samp = fft_processor._create_input_df(data)
df_nlllog_orig['Source'] = 'Human'
df_nlllog_samp['Source'] = 'GPT-4'
df_nlllog = pd.concat([df_nlllog_orig, df_nlllog_samp])

# Log + Z-Score NLL
fft_processor.preprocess = 'logzs'
data = fft_processor._preprocess(nll_pubmed_orig)
df_nlllogzs_orig = fft_processor._create_input_df(data)
data = fft_processor._preprocess(nll_pubmed_samp)
df_nlllogzs_samp = fft_processor._create_input_df(data)
df_nlllogzs_orig['Source'] = 'Human'
df_nlllogzs_samp['Source'] = 'GPT-4'
df_nlllogzs = pd.concat([df_nlllogzs_orig, df_nlllogzs_samp])

# Print some data
print(df_nll.head())
print(df_nlllog.head())
print(df_nlllogzs.head())

150 150
<class 'list'> <class 'list'>
     value  sid Source
0   5.1750    0  Human
1   0.0119    0  Human
2   3.3275    0  Human
3  10.8108    0  Human
4   7.6979    0  Human
      value  sid Source
0  1.820509    0  Human
1  0.011830    0  Human
2  1.464990    0  Human
3  2.469014    0  Human
4  2.163082    0  Human
      value  sid Source
0  1.064695    0  Human
1 -1.214734    0  Human
2  0.616644    0  Human
3  1.881989    0  Human
4  1.496430    0  Human


In [14]:
%%R -i df_nll -i df_nlllog -i df_nlllogzs
require("ggplot2")
require("stringr")

genre <- "pubmed"
est_name <- "mistral"

# NLL
value <- "nll"
p <- ggplot(df_nll, aes(x=value, fill=Source, color=Source)) + geom_density(alpha=0.5) +
    theme_bw() + theme(plot.title = element_text(hjust = 0.5, vjust=-8, size = 12), legend.position=c(0.8,0.8)) +
    labs(x = "NLL", y = "Density")
ggsave(str_interp("${genre}_${est_name}_${value}.pdf"), plot=p, width=5, height=5)

# Log-transformed NLL
value <- "nlllog"
p <- ggplot(df_nlllog, aes(x=value, fill=Source, color=Source)) + geom_density(alpha=0.5) +
    theme_bw() + theme(plot.title = element_text(hjust = 0.5, vjust=-8, size = 12), legend.position=c(0.8,0.8)) +
    labs(x = "log(NLL)", y = "Density")
ggsave(str_interp("${genre}_${est_name}_${value}.pdf"), plot=p, width=5, height=5)

# Log-transformed and Z-Scored NLL
value <- "nlllogzs"
p <- ggplot(df_nlllogzs, aes(x=value, fill=Source, color=Source)) + geom_density(alpha=0.5) +
    theme_bw() + theme(plot.title = element_text(hjust = 0.5, vjust=-8, size = 12), legend.position=c(0.8,0.8)) +
    labs(x = "z-score(log(NLL))", y = "Density")
ggsave(str_interp("${genre}_${est_name}_${value}.pdf"), plot=p, width=5, height=5)